# Authoring Expectation Suites

### Basic setup

In [ ]:
# imports

import os
import great_expectations as gx
import great_expectations.expectations as gxe
from great_expectations.core.expectation_suite import ExpectationSuite

In [ ]:
# constants

asset_name = "trip-data"
batch_definition_name = "partition-by-month"
datasource_name = "postgres"
suite_name = "passenger_count_checker"

In [ ]:
# models

context = gx.get_context(mode="file")

datasource = context.sources.add_postgres(datasource_name, connection_string=os.environ["DB_CONNECTION_STRING"])
data_asset = datasource.add_table_asset(name=asset_name, table_name="trip_data")
batch_definition = data_asset.add_batch_definition(batch_definition_name)
suite = context.suites.add(ExpectationSuite(suite_name))

batch = data_asset.get_batch_list_from_batch_request(batch_definition.build_batch_request())[-1]  # this will get simplified


### Create an expectation

In [ ]:
expectation = gxe.ExpectColumnMinToBeBetween(column="passenger_count", min_value=4, max_value=6)

# see a failure :(
batch.validate(expectation)

### Edit the expectation

In [ ]:
expectation.min_value = 0

# see success!
batch.validate(expectation)


In [ ]:
suite = context.suites.get(suite_name)
suite.add_expectation(expectation)
suite.add_expectation(gxe.ExpectColumnValuesToBeBetween(column="passenger_count", min_value=0, max_value=7))


In [ ]:
# edit an existing expectation in a suite
suite = context.suites.get(suite_name)
expectation = next(e for e in suite.expectations if e.expectation_type == "expect_column_min_to_be_between")
assert isinstance(expectation, gxe.ExpectColumnMinToBeBetween)
expectation.max_value = 0
expectation.save()

print(repr(expectation))

In [ ]:
suite = context.suites.get(suite_name)
expectation = suite.add_expectation(gxe.ExpectColumnValuesToBeBetween(column="passenger_count", min_value=0, max_value=7))
expectation = suite.add_expectation(gxe.ExpectColumnMaxToBeBetween(column="passenger_count", min_value=0, max_value=6))

batch = data_asset.get_batch_list_from_batch_request(batch_definition.build_batch_request())[-1]  # this will get simplified
results = batch.validate(suite)

print(results)